In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import os

In [ ]:
#data_dir = 'data/data-new'
#dat_suffix = ['Osak', 'Glas', 'Edin', 'Toro', 'Melb']

In [ ]:
algo = ['rand', 'pop', 'linreg', 'logreg', 'rank', 'logpwr', 'tranDP', 'tranILP', \
        'ssvm-viterbi', 'ssvm-listViterbi', 'memm']
algo_name = ['Random', 'Popularity', 'POILocationPrediction', 'POIOccurrencePrediction', 'RankSVM', 'RankLogistic', \
             'Markov', 'MarkovPath', 'SSVMViterbi', 'SSVMListViterbi', 'MEMM']
metric_name = ['F$_1$', 'pairs-F$_1$', '$\\tau$']

```dat_ix``` is required in notebook ```shared.ipynb```.

In [ ]:
dat_ix = 0

Run notebook ```shared.ipynb```.

In [ ]:
%run 'shared.ipynb'

Construct file names.

In [ ]:
def build_fnames(dat_ix):
    fnames = []
    for a in algo:
        fnames.append(os.path.join(data_dir, a + '-' + dat_suffix[dat_ix] + '.pkl'))
    return fnames        

In [ ]:
fnames = build_fnames(dat_ix)
fnames

Compute evaluation metrics.

In [ ]:
def calc_metrics(fnames):
    assert(len(fnames) == len(algo))
    recdicts = []
    for f in fnames:
        recdicts.append(pkl.load(open(f, 'rb')))
    
    keys = set(recdicts[0].keys())
    print(len(keys))
    for d in recdicts[1:]:
        keys = keys & set(d.keys())
        print(len(d.keys()))
    print('#Records:', len(keys))
    
    meanvs = np.zeros((len(algo), 3), dtype=np.float)
    stderr = np.zeros((len(algo), 3), dtype=np.float)
    
    for j in range(len(recdicts)):
        d = recdicts[j]
        F1_all = []; pF1_all = []; Tau_all = []
        for q in keys:
            F1, pF1, tau = evaluate(d[q]['PRED'], TRAJ_GROUP_DICT[q])
            F1_all.append(F1); pF1_all.append(pF1); Tau_all.append(tau)
        meanvs[j, 0] = np.mean(F1_all)
        meanvs[j, 1] = np.mean(pF1_all)
        meanvs[j, 2] = np.mean(Tau_all)
        stderr[j, 0] = np.std(F1_all) / np.sqrt(len(F1_all))
        stderr[j, 1] = np.std(pF1_all) / np.sqrt(len(pF1_all))
        stderr[j, 2] = np.std(Tau_all) / np.sqrt(len(Tau_all))
    return meanvs, stderr

In [ ]:
meanvs, stderr = calc_metrics(fnames)

In [ ]:
#meanvs

Generate LaTeX table.

In [ ]:
def gen_latex(meanvs, stderr, title, label):
    assert(meanvs.shape == stderr.shape)
    assert(meanvs.shape == (len(algo), 3))
    
    max_1st = np.zeros(len(metric_name), dtype=np.int)
    max_2nd = np.zeros(len(metric_name), dtype=np.int)
    
    for j in range(meanvs.shape[1]):
        max_2nd[j], max_1st[j] = np.argsort(meanvs[:, j])[-2:]
    
    strs = []
    strs.append('\\begin{table*}[t]\n')
    strs.append('\\caption{' + title + '}\n')
    strs.append('\\label{' + label + '}\n')
    strs.append('\\centering\n')
    strs.append('\\begin{tabular}{l|' + (meanvs.shape[1])*'c' + '} \\hline\n')
    for j in range(meanvs.shape[1]):
        strs.append(' & ' + metric_name[j])
    strs.append(' \\\\ \\hline\n')
    for i in range(meanvs.shape[0]):
        strs.append(algo_name[i] + ' ')
        for j in range(meanvs.shape[1]):
            strs.append('& $')
            if i == max_1st[j]: strs.append('\\mathbf{')
            if i == max_2nd[j]: strs.append('\\mathit{')
            strs.append('%.3f' % meanvs[i, j] + '\\pm' + '%.3f' % stderr[i, j])
            if i in [max_1st[j], max_2nd[j]]: strs.append('}')
            strs.append('$ ')
        strs.append('\\\\\n')
    strs.append('\\hline\n')
    strs.append('\\end{tabular}\n')
    strs.append('\\end{table*}\n')
    return ''.join(strs)    

In [ ]:
strs = gen_latex(meanvs, stderr, 'Performance', 'tab:performance')
print(strs)